In [1]:
project_root_path = '/home/bob/projects/Web3HackerNetwork'

### Load the Commit Statistics
For this clustering we examined 39 GitHub projects containing a total of 2,629 commits. The projects all included the Fluence Marine library. Code for Fluence is generally written in React and Rust.

In [2]:
import sys
bob_lib_path = project_root_path + "/sandbox/python/bob/"
sys.path.append(bob_lib_path)

import commit_stat_engineering
import json

dataset_dir = project_root_path + '/data/github/2022-04-07-marine-search'
project_stats_dir = dataset_dir + '/projects'
all_commit_logs = commit_stat_engineering.find_files('commit-stat.log.json', project_stats_dir)
all_stats = commit_stat_engineering.get_stats_for_all_commits(all_commit_logs)

### Load Labels

In [3]:
import pandas as pd

data = pd.read_csv(project_root_path + '/sandbox/data/bob/commit_labeling.tsv', sep='\t')
data

,Unnamed: 0,wtf,is_bulk,is_standard,is_minor,is_artifact,is_interface,is_javascript,is_rust,cluster,...,textFiles,textLines,javascriptLinePct,rustLinePct,markdownLinePct,jsonLinePct,imgFiles,lockLinePct,yarnLinePct,htmlLinePct
0,e3b155c53bbddf8e02188306a66732785fbdbfcb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,1,5,0.00,0.0,100.00,0.00,0,0.0,0.00,0.0
1,8a4a4d941b83711b509f57de1f51665e0cfdec06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,2,6,0.00,0.0,66.66,0.00,0,0.0,33.33,0.0
2,c07f19718e8ba11e6079ac5347a7c36b24c6f311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,1,5,0.00,0.0,0.00,100.00,0,0.0,0.00,0.0
3,350b4d3352aaec6e2ad9004cbc5a95403c2dfecd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,1,12,0.00,0.0,0.00,0.00,0,0.0,0.00,0.0
4,19e1e198b4db8d5ac98926c1b80e56163ae6b439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,1,2,100.00,0.0,0.00,0.00,0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,ed972d32d8e5b68a3e230080cfc6342ef003420c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,...,14,5495,2.76,0.0,0.00,88.98,0,0.0,0.00,0.0
136,91021d8b4099343049053dc3f0411ed5f59a0167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,...,9,5928,1.46,0.0,0.00,98.06,0,0.0,0.00,0.0
137,90a25bedfeb738a9c16ea6508be733c91758ecd1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,...,23,6760,32.30,0.0,12.64,46.90,0,0.0,0.00,0.0
138,6c291073644710476bd212855558ca9f3aae4a5c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,...,7,10803,0.00,0.0,0.00,99.99,0,0.0,0.00,0.0


### Link Labels to Stats

In [4]:
import numpy as np
def make_x(commit_stats):
    return [
        np.log1p(commit_stats['binFiles']),
        np.log1p(commit_stats['textFiles']),
        np.log1p(commit_stats['textLines']),
        np.log1p(commit_stats['javascriptLines']),
        np.log1p(commit_stats['rustLines']),
        np.log1p(commit_stats['jsonLines']),
        #np.log1p(commit_stats['imgFiles']),
        np.log1p(commit_stats['lockLines']),
        np.log1p(commit_stats['htmlLines'])
    ]


In [5]:
stats = {}
for commit in all_stats:
    stats[commit['commit']] = commit

X = []
Y = []
a = 0
rows = [row for row in data.to_numpy().tolist() if row[4] != 1]
for row in rows: #data.to_numpy() if row['is_artifact'] != 1:
    commit_id = row[0]
    row_labels = row[slice(1, 4)]
    x = make_x(stats[commit_id])
    #print(x)
    #print(a)
    X.append(make_x(stats[commit_id]))
    Y.append(row_labels)#.tolist())
    a += 1
#print(a)

#print(Y[slice(2)])
#print(Y[0][0])
#print(X[slice(2)])

In [6]:
from sklearn.datasets import make_multilabel_classification

sample_X, sample_y = make_multilabel_classification(n_classes=3, random_state=0)
print(sample_y[slice(2)])
print(Y[slice(2)])

[[0 1 0]
 [0 1 0]]
[[nan, nan, nan], [nan, nan, nan]]


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier

print(len(X))
print(len(Y))

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(7, 8), random_state = 1)
clf.fit(X, Y)
score = clf.score(X, Y)
Y_pred = clf.predict(X)
# print(score)
# print(Y_pred)

140
140


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = \
    train_test_split(X, Y, test_size=0.33, random_state=3)

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 6), random_state = 1)
clf.fit(X_train, Y_train)
score = clf.score(X_test, Y_test)
# Y_pred = clf.predict(X)
print(score)
# print(Y_pred)

In [ ]:
#print(X_test)
#print(Y_pred)
pairs = []
for i in range(0,len(X_test)):
    #print(Y_pred[i].tolist())
    pairs.append(Y_pred[i].tolist() + X_test[i])
# print(pairs)

print(len(X_train))
import pandas as pd
pd.DataFrame(pairs).to_csv("/home/bob/test.tsv", sep="\t")